## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.2906,-156.7887,-27.38,68,40,8.05,haze
1,1,Busselton,AU,-33.6500,115.3333,66.09,52,0,4.16,clear sky
2,2,Cidreira,BR,-30.1811,-50.2056,72.81,96,100,4.00,overcast clouds
3,3,Orotukan,RU,62.2667,151.7000,-28.23,100,86,3.60,overcast clouds
4,4,Damietta,EG,31.4165,31.8133,55.29,75,54,17.07,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Cidreira,BR,-30.1811,-50.2056,72.81,96,100,4.00,overcast clouds
6,6,Bambous Virieux,MU,-20.3428,57.7575,79.21,94,75,16.11,moderate rain
7,7,Dingle,PH,10.9995,122.6711,73.44,92,100,10.18,overcast clouds
12,12,Grand Gaube,MU,-20.0064,57.6608,79.38,84,100,8.99,overcast clouds
13,13,Arraial Do Cabo,BR,-22.9661,-42.0278,82.36,78,40,11.50,scattered clouds
14,14,Sola,VU,-13.8833,167.5500,81.77,77,100,25.79,heavy intensity rain
17,17,Kapaa,US,22.0752,-159.3190,78.48,70,0,1.01,clear sky
20,20,Rikitea,PF,-23.1203,-134.9692,78.03,71,9,8.08,light rain
23,23,Yeppoon,AU,-23.1333,150.7333,71.38,73,98,19.44,overcast clouds
24,24,Batouri,CM,4.4333,14.3667,74.23,22,78,2.35,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                242
City                   242
Country                241
Lat                    242
Lng                    242
Max Temp               242
Humidity               242
Cloudiness             242
Wind Speed             242
Current Description    242
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                241
City                   241
Country                241
Lat                    241
Lng                    241
Max Temp               241
Humidity               241
Cloudiness             241
Wind Speed             241
Current Description    241
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cidreira,BR,72.81,overcast clouds,-30.1811,-50.2056,
6,Bambous Virieux,MU,79.21,moderate rain,-20.3428,57.7575,
7,Dingle,PH,73.44,overcast clouds,10.9995,122.6711,
12,Grand Gaube,MU,79.38,overcast clouds,-20.0064,57.6608,
13,Arraial Do Cabo,BR,82.36,scattered clouds,-22.9661,-42.0278,
14,Sola,VU,81.77,heavy intensity rain,-13.8833,167.5500,
17,Kapaa,US,78.48,clear sky,22.0752,-159.3190,
20,Rikitea,PF,78.03,light rain,-23.1203,-134.9692,
23,Yeppoon,AU,71.38,overcast clouds,-23.1333,150.7333,
24,Batouri,CM,74.23,broken clouds,4.4333,14.3667,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
        
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(50)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cidreira,BR,72.81,overcast clouds,-30.1811,-50.2056,Hotel Castelo
6,Bambous Virieux,MU,79.21,moderate rain,-20.3428,57.7575,Casa Tia Villa
7,Dingle,PH,73.44,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
12,Grand Gaube,MU,79.38,overcast clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
13,Arraial Do Cabo,BR,82.36,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
14,Sola,VU,81.77,heavy intensity rain,-13.8833,167.5500,Leumerus Bungalows
17,Kapaa,US,78.48,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
20,Rikitea,PF,78.03,light rain,-23.1203,-134.9692,People ThankYou
23,Yeppoon,AU,71.38,overcast clouds,-23.1333,150.7333,Villa Mar Colina
24,Batouri,CM,74.23,broken clouds,4.4333,14.3667,Hotel Hollywood


In [12]:
# replace empty values for Hotel Name with NaN
import numpy as np

hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace("", np.nan)
hotel_df["Hotel Name"].isnull().sum()

8

In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(50)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Cidreira,BR,72.81,overcast clouds,-30.1811,-50.2056,Hotel Castelo
6,Bambous Virieux,MU,79.21,moderate rain,-20.3428,57.7575,Casa Tia Villa
7,Dingle,PH,73.44,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
12,Grand Gaube,MU,79.38,overcast clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
13,Arraial Do Cabo,BR,82.36,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
14,Sola,VU,81.77,heavy intensity rain,-13.8833,167.5500,Leumerus Bungalows
17,Kapaa,US,78.48,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
20,Rikitea,PF,78.03,light rain,-23.1203,-134.9692,People ThankYou
23,Yeppoon,AU,71.38,overcast clouds,-23.1333,150.7333,Villa Mar Colina
24,Batouri,CM,74.23,broken clouds,4.4333,14.3667,Hotel Hollywood


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))